# Kanun Teklifleri - Data Collection

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException

import pandas as pd
import time
import os

In [2]:
driver = webdriver.Chrome()
driver.implicitly_wait(30)

## Kanun Teklifleri

- source_page: https://www.tbmm.gov.tr/Yasama/Gecmis-Kanun-Teklifleri
- donem_values:
    - 22. Dönem Tüm Yasama Yılları: **220**
    - 23. Dönem Tüm Yasama Yılları: **230**
    - 24. Dönem Tüm Yasama Yılları: **240**
    - 25. Dönem Tüm Yasama Yılları: **250**
    - 26. Dönem Tüm Yasama Yılları: **260**
- select_donem (Dönemi ve Yasama Yılı) XPATH: //*[@id="DonemYasamaYili"]
- input_search (Sorguyu Başlat) XPATH: //*[@id="main-area"]/form/div[10]/div/input

In [3]:
def get_sonuc_page(source_page, donem):

    # Open source page
    driver.get(source_page)
    
    # Select Dönem
    select_donem = Select(driver.find_element(By.XPATH, """//*[@id="DonemYasamaYili"]"""))
    donem = str(donem) + "0"
    select_donem.select_by_value(donem)
    
    # Find "Sorguyu Başlat" button
    input_search = driver.find_element(By.XPATH, """//*[@id="main-area"]/form/div[10]/div/input""")

    # Click "Sorguyu Başlat" button
    input_search.click()
    
    # Get total page size
    pagination = driver.find_element(By.XPATH, """//*[@id="dataTable_paginate"]/ul""")
    pagination_elements = pagination.find_elements(By.TAG_NAME, "li")
    
    # if there are no records:
    if len(pagination_elements) == 2:
        page_size = 0
    # if there is one page
    elif len(pagination_elements) == 3:
        page_size = 1
    # if there are two pages:
    elif len(pagination_elements) == 4:
        page_size = 2
    # if there are more than two pages
    else:
        page_size = int(pagination_elements[-2].text)
    
    return page_size

In [4]:
def collect_urls():
    
    urls = []
    
    # Find table
    table = driver.find_element(By.XPATH, """//*[@id="dataTable"]/tbody""")
    
    # Get rows of table
    rows = None
    while rows is None:
        try:
            rows = table.find_elements(By.TAG_NAME, "tr")
        except:
            pass
        
    for i, row in enumerate(rows):
        url = row.find_elements(By.TAG_NAME, "td")[3].find_elements(By.TAG_NAME, "div")[4].find_element(By.TAG_NAME, "a").get_attribute("href")
        urls.append(url)
        
    return urls

In [5]:
donemler = [22, 23, 24, 25, 26]
source_page = "https://www.tbmm.gov.tr/Yasama/Gecmis-Kanun-Teklifleri"

In [6]:
# Create urls folder if not exists
if "urls" not in os.listdir():
    os.mkdir("urls")

# Write urls to txt file
with open("urls/kanun_teklifi_urls.txt", "a+") as f:
    for donem in donemler:
        # Get sonuc page & page size
        page_size = get_sonuc_page(source_page, donem)
        print(f"Dönem: {donem} | Page Size: {page_size} ... ", end="")
    
        page = 1
        
        while page <= page_size:
            try:
                time.sleep(1)
                # Collect urls in the page
                for url in collect_urls():
                    f.write(f"{url}\n")
                
                # Find "Sonraki" button
                next_button = driver.find_element(By.XPATH, """//*[@id="dataTable_next"]""")
                
                # Click "Sonraki" button
                next_button.click()
                
                page += 1
                
            except:
                time.sleep(5)
                continue
        print("Completed!")

Dönem: 22 | Page Size: 49 ... Completed!
Dönem: 23 | Page Size: 40 ... Completed!
Dönem: 24 | Page Size: 78 ... Completed!
Dönem: 25 | Page Size: 18 ... Completed!
Dönem: 26 | Page Size: 67 ... Completed!


In [7]:
# Read urls from txt file
with open("urls/kanun_teklifi_urls.txt", "r") as f:
    urls = [url.strip() for url in f.readlines()]

In [8]:
def get_tables(url):
    
    driver.get(url)
    
    main_area = driver.find_element(By.XPATH, """//*[@id="main-area"]""")
    
    kanun_teklifi_bilgileri_df = pd.DataFrame()
    kanun_teklifi_imza_sahipleri_df = pd.DataFrame()
    
    
    for i in range(len(main_area.find_elements(By.TAG_NAME, "div"))):
        if main_area.find_elements(By.TAG_NAME, "span")[i].text == "Kanun Teklifi Bilgileri":
            kanun_teklifi_bilgileri = main_area.find_elements(By.TAG_NAME, "div")[i]
            kanun_teklifi_bilgileri_tr_tags = kanun_teklifi_bilgileri.find_elements(By.TAG_NAME, "tr")
            kanun_teklifi_bilgileri_df = pd.DataFrame(pd.Series({kanun_teklifi_bilgileri_tr_tags[i].find_elements(By.TAG_NAME, "td")[0].text: kanun_teklifi_bilgileri_tr_tags[i].find_elements(By.TAG_NAME, "td")[1].text for i in range(1, len(kanun_teklifi_bilgileri_tr_tags))})).T
            kanun_teklifi_id = ''.join(kanun_teklifi_bilgileri_df.iloc[0, 0].split(" / ") + kanun_teklifi_bilgileri_df.iloc[0, 1].split(" / "))
            kanun_teklifi_bilgileri_df.columns = ["donemi_ve_yasama_yili", "esas_numarasi", "baskanliga_gelis_tarihi", "teklifin_basligi", "teklifin_ozeti", "son_durumu", "teklifin_sonucu"]
            kanun_teklifi_bilgileri_df["kanun_teklifi_id"] = kanun_teklifi_id
            
        if main_area.find_elements(By.TAG_NAME, "span")[i].text == "Kanun Teklifi İmza Sahipleri":
            kanun_teklifi_imza_sahipleri = main_area.find_elements(By.TAG_NAME, "div")[i]
            kanun_teklifi_imza_sahipleri_tr_tags = kanun_teklifi_imza_sahipleri.find_elements(By.TAG_NAME, "tr")

            header = kanun_teklifi_imza_sahipleri_tr_tags[0].find_elements(By.TAG_NAME, "th")
            header_items = [item.text for item in header]
            header_items[-1] = "İlk İmza"

            adi_ve_soyadi = [kanun_teklifi_imza_sahipleri_tr_tags[i].find_elements(By.TAG_NAME, "td")[0].text for i in range(1, len(kanun_teklifi_imza_sahipleri_tr_tags))]
            partisi = [kanun_teklifi_imza_sahipleri_tr_tags[i].find_elements(By.TAG_NAME, "td")[1].text for i in range(1, len(kanun_teklifi_imza_sahipleri_tr_tags))]
            secim_cevresi = [kanun_teklifi_imza_sahipleri_tr_tags[i].find_elements(By.TAG_NAME, "td")[2].text for i in range(1, len(kanun_teklifi_imza_sahipleri_tr_tags))]
            ilk_imza = [kanun_teklifi_imza_sahipleri_tr_tags[i].find_elements(By.TAG_NAME, "td")[3].text for i in range(1, len(kanun_teklifi_imza_sahipleri_tr_tags))]

            kanun_teklifi_imza_sahipleri_df = pd.DataFrame([adi_ve_soyadi, partisi, secim_cevresi, ilk_imza]).T.rename({i:header_items[i] for i in range(4)}, axis=1)
            kanun_teklifi_imza_sahipleri_df.columns = ["adi_ve_soyadi", "partisi", "secim_cevresi", "ilk_imza"]
            kanun_teklifi_imza_sahipleri_df["kanun_teklifi_id"] = kanun_teklifi_id
            kanun_teklifi_imza_sahipleri_df["ilk_imza"] = kanun_teklifi_imza_sahipleri_df["ilk_imza"].apply(lambda x: True if x == "İlk İmza Sahibi" else False)

    return kanun_teklifi_bilgileri_df, kanun_teklifi_imza_sahipleri_df

In [9]:
if "kanun_teklifleri" not in os.listdir():
    os.mkdir("kanun_teklifleri")

# Create csv files and headers
pd.DataFrame(columns=["donemi_ve_yasama_yili",
                      "esas_numarasi",
                      "baskanliga_gelis_tarihi",
                      "teklifin_basligi",
                      "teklifin_ozeti",
                      "son_durumu",
                      "teklifin_sonucu",
                      "kanun_teklifi_id"]).to_csv(f"kanun_teklifleri/kanun_teklifleri.csv",
                                                   mode="w", index=False)

pd.DataFrame(columns=["adi_ve_soyadi",
                      "partisi",
                      "secim_cevresi",
                      "ilk_imza",
                      "kanun_teklifi_id"]).to_csv(f"kanun_teklifleri/kanun_teklifi_imza_sahipleri.csv",
                                                   mode="w", index=False)

In [10]:
# Write to csv files
for i, url in enumerate(urls):
    if i % 500 == 0:
        print(f"{i}/{len(urls)} | {(i)/len(urls)*100:.2f}%")
    kanun_teklifi, kanun_teklifi_imza_sahipleri = get_tables(url)
    kanun_teklifi.to_csv(f"kanun_teklifleri/kanun_teklifleri.csv", mode="a+", index=False, header=False)
    kanun_teklifi_imza_sahipleri.to_csv(f"kanun_teklifleri/kanun_teklifi_imza_sahipleri.csv", mode="a+", index=False, header=False)
print(f"{len(urls)}/{len(urls)} | {(len(urls))/len(urls)*100:.2f}%")

0/12437 | 0.00%
500/12437 | 4.02%
1000/12437 | 8.04%
1500/12437 | 12.06%
2000/12437 | 16.08%
2500/12437 | 20.10%
3000/12437 | 24.12%
3500/12437 | 28.14%
4000/12437 | 32.16%
4500/12437 | 36.18%
5000/12437 | 40.20%
5500/12437 | 44.22%
6000/12437 | 48.24%
6500/12437 | 52.26%
7000/12437 | 56.28%
7500/12437 | 60.30%
8000/12437 | 64.32%
8500/12437 | 68.34%
9000/12437 | 72.36%
9500/12437 | 76.38%
10000/12437 | 80.41%
10500/12437 | 84.43%
11000/12437 | 88.45%
11500/12437 | 92.47%
12000/12437 | 96.49%
12437/12437 | 100.00%


In [11]:
driver.quit()